In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By

from os import path, rename
import shutil
import re
import random
import webbrowser


In [2]:
# treat empty strings and np.nan as NAm will impact dropna method
pd.options.mode.use_inf_as_na = True
# set Jupyter to display all output from a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
MAX_USER_PROFILES = 30

In [ ]:
### setup Google Chrome Driver (must match current browser version)
# goto https://chromedriver.chromium.org/downloads and download appropriate chromedriver version
#mv chromedriver /usr/local/bin
#sudo chmod a+x /usr/local/bin/chromedriver
# make it trustable
#xattr -d com.apple.quarantine  /usr/local/bin/chromedriver
#/usr/local/bin/chromedriver --version 
## now run it in terminal and use existing chrome profile
# "Google Chrome" --remote-debugging-port=9222 --user-data-dir="/Users/rajivdulepet/Library/Application Support/Google/Chrome" "https://google.com"


In [4]:
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
#Change chrome driver path accordingly
chrome_driver = "chromedriver"
driver = webdriver.Chrome(chrome_driver, chrome_options=chrome_options)

#time.sleep(2)
#driver.execute_script("window.open('');")
#time.sleep(2)

/var/folders/x9/8gyw12bj7b72_vtgyj_q2jkr0000gn/T/ipykernel_94235/2649563906.py:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver, chrome_options=chrome_options)


In [5]:
def loadNCIDictionaryPage(pageDriver, url, timeout=10):
    pageDriver.switch_to.window(pageDriver.window_handles[1])
    time.sleep(timeout)
    pageDriver.get(url)
    time.sleep(timeout)


In [6]:
import re
def extractCancerTermsDefinitions(pageDriver, listStruct):
    dictionaryTerms = pageDriver.find_elements(By.XPATH, '//*[@id="NCI-glossary-app-root"]/div[2]/dl/div')
    for term_def in dictionaryTerms:
        #//*[@id="NCI-glossary-app-root"]/div[2]/dl/div[1]/dt/dfn/a
        term_name = term_def.find_element(By.XPATH, './/dt/dfn/a')
        term_desc = term_def.find_element(By.XPATH, './/dd[contains(@class,"definition")]')
        term_desc_text = term_desc.text
        
        listStruct.append({'term': term_name.text, 'definition': term_desc_text, 'url': pageDriver.current_url})


In [7]:
import re
def extractDrugTermsDefinitions(pageDriver, listStruct):
    dictionaryTerms = pageDriver.find_elements(By.XPATH, '//*[@id="NCI-dd-app-root"]/div[2]/dl/dt')
    for term_def in dictionaryTerms:
        #//*[@id="NCI-glossary-app-root"]/div[2]/dl/div[1]/dt/dfn/a
        term_name = term_def.find_element(By.XPATH, './/dfn/a')
        term_desc = term_def.find_element(By.XPATH, './following-sibling::dd')
        term_desc_text = term_desc.text
        
        listStruct.append({'term': term_name.text, 'definition': term_desc_text, 'url': pageDriver.current_url})
# //*[@id="NCI-dd-app-root"]/div[2]/dl/dt[1]

In [16]:
%%time
import string

#listCancerTermsDefinitions = []
base_nci_term_def_url = 'https://www.cancer.gov/publications/dictionaries/cancer-terms/expand/'
# last part of URL that we need to append on fly
last_url_parts = list(string.ascii_uppercase)
last_url_parts.append('#')

target_urls = [base_nci_term_def_url + last_url_part for last_url_part in last_url_parts]
target_urls = ['https://www.cancer.gov/publications/dictionaries/cancer-terms/expand/%23']

for target_url in target_urls:
    print(target_url)
    loadNCIDictionaryPage(driver, target_url)
    extractCancerTermsDefinitions(driver, listCancerTermsDefinitions)


https://www.cancer.gov/publications/dictionaries/cancer-terms/expand/%23
CPU times: user 200 ms, sys: 14.6 ms, total: 214 ms
Wall time: 22.3 s


In [14]:
%%time
import string

#listCancerDrugDefinitions = []
base_nci_term_def_url = 'https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/'
# last part of URL that we need to append on fly
last_url_parts = list(string.ascii_uppercase)
last_url_parts.append('#')

target_urls = [base_nci_term_def_url + last_url_part for last_url_part in last_url_parts]
target_urls = ['https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/%23']

for target_url in target_urls:
    print(target_url)
    loadNCIDictionaryPage(driver, target_url, 20)
    extractDrugTermsDefinitions(driver, listCancerDrugDefinitions)


https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/%23
CPU times: user 131 ms, sys: 11.7 ms, total: 143 ms
Wall time: 42.3 s


In [17]:
len(listCancerDrugDefinitions)
len(listCancerTermsDefinitions)

9066

9053

In [19]:
dfCancerDrugDefinitions = pd.DataFrame(listCancerDrugDefinitions)
dfCancerTermsDefinitions = pd.DataFrame(listCancerTermsDefinitions)
dfCancerDrugDefinitions.columns
dfCancerDrugDefinitions.shape
dfCancerTermsDefinitions.columns
dfCancerTermsDefinitions.shape

Index(['term', 'definition', 'url'], dtype='object')

(9066, 3)

Index(['term', 'definition', 'url'], dtype='object')

(9053, 3)

In [21]:
dfCancerDrugDefinitions['url'].unique()

array(['https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/A',
       'https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/B',
       'https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/C',
       'https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/D',
       'https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/E',
       'https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/F',
       'https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/G',
       'https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/H',
       'https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/I',
       'https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/J',
       'https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/K',
       'https://www.cancer.gov/publications/dictionaries/cancer-drug/expand/L',
       'https://www.cancer.gov/publicati

In [22]:
dfCancerDrugDefinitions.to_csv('dfCancerDrugDefinitions.csv', index=False)
dfCancerTermsDefinitions.to_csv('dfCancerTermsDefinitions.csv', index=False)

In [25]:
dfCancerTermsDefinitions.sample(5)

,term,definition,url
2451,early-stage cancer,A term used to describe cancer that is early i...,https://www.cancer.gov/publications/dictionari...
6019,pegfilgrastim,A drug that is a form of filgrastim and is use...,https://www.cancer.gov/publications/dictionari...
8710,vincristine sulfate,A drug used to treat acute leukemia and is som...,https://www.cancer.gov/publications/dictionari...
2952,fexofenadine,A drug used to treat certain allergy symptoms....,https://www.cancer.gov/publications/dictionari...
2343,docetaxel-cisplatin-fluorouracil,A chemotherapy combination used to treat certa...,https://www.cancer.gov/publications/dictionari...


In [34]:
dfCancerTermsDefinitions[dfCancerTermsDefinitions.term.str.contains('medull')]

,term,definition,url
172,adrenal medulla,The inner part of the adrenal gland (a small o...,https://www.cancer.gov/publications/dictionari...
2863,extramedullary myeloid tumor,A rare type of cancer that is made up of myelo...,https://www.cancer.gov/publications/dictionari...
2902,familial medullary thyroid cancer,An inherited form of medullary thyroid cancer ...,https://www.cancer.gov/publications/dictionari...
4853,medullary breast carcinoma,A rare type of breast cancer that often can be...,https://www.cancer.gov/publications/dictionari...
4854,medullary thyroid cancer,Cancer that develops in C cells of the thyroid...,https://www.cancer.gov/publications/dictionari...
4855,medulloblastoma,A fast-growing type of cancer that forms in th...,https://www.cancer.gov/publications/dictionari...
7541,stage I medullary thyroid cancer,Cancer is found in the thyroid only and the tu...,https://www.cancer.gov/publications/dictionari...
7601,stage II medullary thyroid cancer,One of the following is found: (1) cancer is f...,https://www.cancer.gov/publications/dictionari...
7663,stage III medullary thyroid cancer,The tumor is any size and cancer may have spre...,https://www.cancer.gov/publications/dictionari...
7729,stage IV medullary thyroid cancer,"Stage IV is divided into stages IVA, IVB, and ...",https://www.cancer.gov/publications/dictionari...


In [33]:
dfCancerTermsDefinitions[dfCancerTermsDefinitions.term.str.contains('adrenal medulla')]['definition'].values

array(['The inner part of the adrenal gland (a small organ on top of each kidney). The adrenal medulla makes chemicals such as epinephrine (adrenaline) and norepinephrine (noradrenaline) which are involved in sending nerve signals.'],
      dtype=object)